In [ ]:
"""Valuation with recipes

This notebook shows how to value a portfolio using recipes with different pricing sources

Attributes
----------
valuation
transactions
recipes
manifests
"""

# Valuation with recipes

This notebooks shows how to value a portfolio using recipes with different pricing sources

## Setup LUSID

# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

In [ ]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

## Load source data

Create a scope and portfolio code

In [ ]:
scope = "valuation-sample"
portfolio_code = "EQUITY_UK"

In [ ]:
df = pd.read_csv("data/equity_transactions.csv")
df

## Load instruments

Create the instruments and add on ISIN and Sedol as identifiers

In [ ]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ISIN",
        "Sedol": "sedol",
    },
    "required": {
        "name": "name"
    },
}

In [ ]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

## Create portfolio

In [ ]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [ ]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

## Load in transactions

In [ ]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

In [ ]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)
    
succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

## Load in quotes

Load the source quotes containing bid, mid and ask

In [ ]:
quotes_df = pd.read_csv("data/equity_quotes.csv")
quotes_df

In [ ]:
price_fields = ["bid", "mid", "ask"]

quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$Isin",
    "quote_id.effective_at": "quote_date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "ISIN",
    "metric_value.unit": "$GBP",
}

In [ ]:
for price_field in price_fields:
    
    quotes_mapping["quote_id.quote_series_id.field"] = f"${price_field}"
    quotes_mapping["metric_value.value"] = price_field

    result = load_from_data_frame(
        api_factory = api_factory,
        scope=scope,
        data_frame=quotes_df,
        mapping_required=quotes_mapping,
        mapping_optional={},
        file_type="quotes"
    )

    succ, failed, errors = format_quotes_response(result)
    display(pd.DataFrame(data=[{f"[{price_field}] success": len(succ), "failed": len(failed), "errors": len(errors)}]))

## Valuation

Perform a valuation on the portfolio

In [ ]:
def aggregation_request(valuation_effectiveAt, price_field):
    inline_recipe = models.ConfigurationRecipe(
        scope="User",
        code='valuation_recipe',
        market=models.MarketContext(
            market_rules=[
                
                # define how to resolve the quotes
                models.MarketDataKeyRule(            
                    key='Equity.Isin.*',
                    supplier='Lusid',
                    data_scope=scope,
                    quote_type='Price',
                    field=price_field
                ),
            ],
            options=models.MarketOptions(
                default_supplier='Lusid',
                default_instrument_code_type='Isin',
                default_scope=scope,
                manifest_level_of_detail="Full"
            )
        ),
        pricing=models.PricingContext(
            options={
                "AllowPartiallySuccessfulEvaluation": True
            },
        ),
    )

    return models.AggregationRequest(
        inline_recipe=inline_recipe,
        metrics=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Holding/default/PV", "Proportion"),
            models.AggregateSpec("Holding/default/PV", "Sum"),
            models.AggregateSpec("Holding/default/Units", "Sum"),
            models.AggregateSpec("Holding/default/Error", "Value")
        ],
        group_by=["Instrument/default/Name"],
        
        # choose the valuation time for the request        
        effective_at=valuation_effectiveAt
    )

Run a valuation using a recipe using "mid" quotes

In [ ]:
aggregation_api = api_factory.build(lusid.AggregationApi)
aggregation = aggregation_api.get_aggregation(scope=scope,
                                                           code=portfolio_code,
                                                           aggregation_request=aggregation_request("2020-04-24T00:00:00.000Z", "mid"))
pd.DataFrame(aggregation.data)

Run a valuation using a recipe using "bid" quotes

In [ ]:
aggregation_api = api_factory.build(lusid.AggregationApi)
aggregation = aggregation_api.get_aggregation(scope=scope,
                                                           code=portfolio_code,
                                                           aggregation_request=aggregation_request("2020-04-24T00:00:00.000Z", "bid"))
pd.DataFrame(aggregation.data)

View the manifest, this shows where the data used in the valuation originated from

In [ ]:
manifest_df = pd.DataFrame()
for manifest in aggregation.manifests:
    for manifest_quote in manifest.quotes:
        quote_df = lusid_response_to_data_frame([manifest_quote.quote])
        quote_df.insert(0, "market_data_rule", manifest_quote.rule)
        manifest_df = manifest_df.append(quote_df)

display(manifest_df)